<a href="https://colab.research.google.com/github/omgits0mar/Datasets/blob/master/model1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import jupyterthemes as jt
# from jupyterthemes import get_themes
# import jupyterthemes as jt
# from jupyterthemes.stylefx import set_nb_theme

import jupyterthemes as jt
from jupyterthemes.stylefx import set_nb_theme

In [2]:
# set_nb_theme('onedork')
# #monokai
# #chesterish
# #oceans16 gamed
# #onedork gamed brdo
# #solarizedl

# Code start from here :

### Importing :

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import datetime

from sklearn.metrics import mean_squared_error, r2_score
from sklearn.tree import DecisionTreeRegressor
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBRegressor


### Reading data from csv

In [4]:
df = pd.read_csv('airline-price-prediction.csv')

In [5]:
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price
0,5/3/2022,Vistara,UK,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...","56,588"
1,18-03-2022,Vistara,UK,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}","59,649"
2,9/3/2022,GO FIRST,G8,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}","3,916"
3,15-03-2022,GO FIRST,G8,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}","6,031"
4,22-03-2022,Vistara,UK,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}","7,081"


In [6]:
df.shape

(240208, 11)

### Data preprocessing on 'price'

In [7]:
df['price'] = df['price'].str.replace(",", "")
df['price'] = pd.to_numeric(df['price'])

### Data preprocessing on 'date'

In [8]:
df["date"]=pd.to_datetime(df["date"])
df["date"]=df["date"].dt.strftime("%m/%d/%Y")


In [9]:
print(df[df.columns[0]])

0         05/03/2022
1         03/18/2022
2         09/03/2022
3         03/15/2022
4         03/22/2022
             ...    
240203    03/25/2022
240204    02/15/2022
240205    02/23/2022
240206    11/02/2022
240207    02/18/2022
Name: date, Length: 240208, dtype: object


In [10]:
pd.DatetimeIndex(df["date"]).weekday

Int64Index([1, 4, 5, 1, 1, 6, 0, 0, 0, 1,
            ...
            3, 3, 0, 2, 3, 4, 1, 2, 2, 4],
           dtype='int64', name='date', length=240208)

In [11]:
df['month'] = pd.DatetimeIndex(df['date']).month
df['day'] = pd.DatetimeIndex(df['date']).day
df['dayofyear'] = pd.DatetimeIndex(df['date']).dayofyear

In [12]:
#df = pd.get_dummies(df, columns=['month'], drop_first=True, prefix='month')

### Data preprocessing : 'ch_code'

In [13]:
lbl_enc = LabelEncoder()
df['ch_code'] = lbl_enc.fit_transform(df[["ch_code"]])
print(df['ch_code'])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0         7
1         7
2         3
3         3
4         7
         ..
240203    2
240204    2
240205    1
240206    7
240207    6
Name: ch_code, Length: 240208, dtype: int64


### Data preprocessing : 'dep_time'

In [14]:
df["dep_time"]=pd.to_datetime(df["dep_time"])
df['dep_time'] = df['dep_time'].dt.strftime("%-H:%M")

In [15]:
df["dep_hour"]=pd.DatetimeIndex(df["dep_time"]).hour
df["dep_minute"]=pd.DatetimeIndex(df["dep_time"]).minute

### Data preprocessing : 'time_taken'

In [16]:
df["hours_taken"] = df["time_taken"].str.split('h').str.get(0)
df["minutes_taken"] = df["time_taken"].str[4:6]
df["minutes_taken"] = df["minutes_taken"].str.replace('m', '')
df["minutes_taken"] = df["minutes_taken"].str.replace('h', '')
df["hours_taken"] = pd.to_numeric(df["hours_taken"])
df["minutes_taken"] = pd.to_numeric(df["minutes_taken"], errors='coerce')
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...",56588,5,3,123,9,45,10.0,10.0
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}",59649,3,18,77,5,45,6.0,30.0
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",3916,9,3,246,14,30,8.0,10.0
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}",6031,3,15,74,10,40,9.0,40.0
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1-stop\n\t\t\t\t\t\t\t\t\t\t\t\t\n\t\t\t\t\t\t...,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}",7081,3,22,81,12,30,7.0,25.0


### Data preprocessing : 'stop'

In [17]:
df["stop"] = df["stop"].str.split('-').str.get(0)
df["stop"] = df["stop"].replace(['non'], 0)
df.isna().sum() #  28944 null vals
df["stop"] = df["stop"].replace(['2+'], 2) # Indicates for 2 or more stops
df['stop'] = df['stop'].fillna(0)
df['stop'] = pd.to_numeric(df['stop'])
# print(df[9:14])

### Data preprocessing : 'arr_time'

In [18]:
df["arr_time"]=pd.to_datetime(df["arr_time"])
df['arr_time'] = df['arr_time'].dt.strftime("%-H:%M")
df["arr_hour"]=pd.DatetimeIndex(df["arr_time"]).hour
df["arr_minute"]=pd.DatetimeIndex(df["arr_time"]).minute
df["arr_hour"] = pd.to_numeric(df["arr_hour"])
df["arr_minute"] = pd.to_numeric(df["arr_minute"])
df.head()


,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,price,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken,arr_hour,arr_minute
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1,19:55,business,"{'source': 'Bangalore', 'destination': 'Hydera...",56588,5,3,123,9,45,10.0,10.0,19,55
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1,12:15,business,"{'source': 'Delhi', 'destination': 'Hyderabad'}",59649,3,18,77,5,45,6.0,30.0,12,15
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1,22:40,economy,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",3916,9,3,246,14,30,8.0,10.0,22,40
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1,20:20,economy,"{'source': 'Mumbai', 'destination': 'Delhi'}",6031,3,15,74,10,40,9.0,40.0,20,20
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1,19:55,economy,"{'source': 'Chennai', 'destination': 'Kolkata'}",7081,3,22,81,12,30,7.0,25.0,19,55


### Data preprocessing : 'type'

In [19]:
df['type'] = lbl_enc.fit_transform(df[["type"]])
print(df['type'])

0         0
1         0
2         1
3         1
4         1
         ..
240203    1
240204    0
240205    1
240206    1
240207    1
Name: type, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


### Data preprocessing : 'route'

In [20]:
df['source'] = df['route'].str.split( ', ').str.get(0).str.split(':').str.get(1)
df['destination'] = df['route'].str.split( ', ').str.get(1).str.split(':').str.get(1).str.split('}').str.get(0)
df['source'] = df['source'].str.replace('\'', "")
df['destination'] = df['destination'].str.replace('\'', "")

In [21]:
df.head()

,date,airline,ch_code,num_code,dep_time,time_taken,stop,arr_time,type,route,...,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken,arr_hour,arr_minute,source,destination
0,05/03/2022,Vistara,7,812,9:45,10h 10m,1,19:55,0,"{'source': 'Bangalore', 'destination': 'Hydera...",...,3,123,9,45,10.0,10.0,19,55,Bangalore,Hyderabad
1,03/18/2022,Vistara,7,975,5:45,06h 30m,1,12:15,0,"{'source': 'Delhi', 'destination': 'Hyderabad'}",...,18,77,5,45,6.0,30.0,12,15,Delhi,Hyderabad
2,09/03/2022,GO FIRST,3,7537,14:30,08h 10m,1,22:40,1,"{'source': 'Mumbai', 'destination': 'Hyderabad'}",...,3,246,14,30,8.0,10.0,22,40,Mumbai,Hyderabad
3,03/15/2022,GO FIRST,3,287,10:40,09h 40m,1,20:20,1,"{'source': 'Mumbai', 'destination': 'Delhi'}",...,15,74,10,40,9.0,40.0,20,20,Mumbai,Delhi
4,03/22/2022,Vistara,7,826,12:30,07h 25m,1,19:55,1,"{'source': 'Chennai', 'destination': 'Kolkata'}",...,22,81,12,30,7.0,25.0,19,55,Chennai,Kolkata


In [22]:
df['source'] = lbl_enc.fit_transform(df[["source"]])
print(df['source'])

0         0
1         2
2         5
3         5
4         1
         ..
240203    1
240204    2
240205    2
240206    2
240207    4
Name: source, Length: 240208, dtype: int64


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [23]:
df['destination'] = lbl_enc.fit_transform(df[["destination"]])
print(df['destination'])
df = df.dropna()
df = df.drop(['airline', 'date', 'dep_time', "time_taken", 'arr_time', 'route',], axis=1)
df.head()
#df = pd.get_dummies(df)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0         3
1         3
2         3
3         2
4         4
         ..
240203    3
240204    4
240205    0
240206    5
240207    3
Name: destination, Length: 240208, dtype: int64


,ch_code,num_code,stop,type,price,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken,arr_hour,arr_minute,source,destination
0,7,812,1,0,56588,5,3,123,9,45,10.0,10.0,19,55,0,3
1,7,975,1,0,59649,3,18,77,5,45,6.0,30.0,12,15,2,3
2,3,7537,1,1,3916,9,3,246,14,30,8.0,10.0,22,40,5,3
3,3,287,1,1,6031,3,15,74,10,40,9.0,40.0,20,20,5,2
4,7,826,1,1,7081,3,22,81,12,30,7.0,25.0,19,55,1,4


In [24]:
print(df.columns)

Index(['ch_code', 'num_code', 'stop', 'type', 'price', 'month', 'day',
       'dayofyear', 'dep_hour', 'dep_minute', 'hours_taken', 'minutes_taken',
       'arr_hour', 'arr_minute', 'source', 'destination'],
      dtype='object')


In [25]:
# from sklearn.preprocessing import StandardScaler
# ss = StandardScaler()
# df_scaled = pd.DataFrame(ss.fit_transform(df),columns = df.columns)
# df_scaled.head()

In [26]:
X = df.loc[:, df.columns != 'price']
Y = df['price']

In [27]:
dtree = DecisionTreeRegressor(max_depth=20, min_samples_leaf=0.10, random_state=8)

In [28]:
XTrain, XTest, YTrain, YTest = train_test_split(X, Y, test_size=0.2, shuffle=True, random_state=8)

In [29]:
dtree.fit(XTrain, YTrain)

pred_train_tree= dtree.predict(XTrain)
print(np.sqrt(mean_squared_error(YTrain, pred_train_tree)))
print(r2_score(YTrain, pred_train_tree))

pred_test_tree= dtree.predict(XTest)
print(np.sqrt(mean_squared_error(YTest, pred_test_tree)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, pred_test_tree))
print(r2_score(YTest, pred_test_tree))

7341.735485727376
0.8951801619904182
7349.826148562579
Mean Square Error for testing set 54019944.41409424
0.895910945352473


In [30]:
polynomial = PolynomialFeatures(degree=3)

XPolynomial = polynomial.fit_transform(XTrain)

Model = linear_model.LinearRegression()
Model.fit(XPolynomial, YTrain)

TrainPrediction = Model.predict(XPolynomial)

TesTPrediction = Model.predict(polynomial.fit_transform(XTest))

print(np.sqrt(mean_squared_error(YTest, TesTPrediction)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, TesTPrediction))
print(r2_score(YTest, TesTPrediction))

5036.0660595933
Mean Square Error for testing set 25361961.356587593
0.9511309644938186


# No dropping && Label encoder
## No Scaling Applied :
### 25,361,961-->Best poly(d=3,r=8)------> Acc = 0.9511
# Drop arr_hour,min,dep_hour,min,dayofyear && Label encoder for all:
## No Scaling Applied :
### 25,063,325 -->poly(d=3,r=120)------> Acc = 0.9512
### 48,657,835 -->Poly(d=5,r=120)------> Acc = 0.90525
### 25,498,894 -->Poly(d=4,r=120)------> Acc = 0.95035
### 24,642,093 -->BEST Poly(d=4,r=8)------> Acc = 0.95251
### 44,714,500 -->ElasticNet(d=2,r=8)------> Acc = 0.91
### 54,019,944-->Dtree-------> Acc = 0.89
## Applied Feature Scaling (Not sure as it implies on price also):
### 0.04603038 -->Poly(d=4,r=8)------> Acc = 0.95430
### 0.10495510 -->Dtree
### 0.47465824 -->ElasticNet(d=2,r=8)------> Acc = 0.4746
# Drop arr_hour,min,dep_hour,min,weekofyear,month,day && Label encoder for all:
  *   0.0447658 -->BEST Poly(d=5,r=8)------> Acc = 0.955561

# Conclusion :
  ### Label encoder > Dummy, OneHot
  ### Polynomial > Linear,Dtree,Ridge,Elastic
  ### Polynomial degree 3 with more features acts nearly Best
# BIG NOTE :
  ### all this data gathered on GoogleCollab and it may slightly differs in jupyter or pycharm

 

In [31]:
import xgboost as xgb

data_dmatrix = xgb.DMatrix(data=X,label=Y)


xg_reg = xgb.XGBRegressor(objective ='reg:squarederror', colsample_bytree = 0.7, learning_rate = 0.001,
                max_depth = 20, alpha = 50, n_estimators = 20)
xg_reg.fit(XTrain,YTrain)

preds = xg_reg.predict(XTest)
print(np.sqrt(mean_squared_error(YTest, preds)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, preds))
print(r2_score(YTest, preds))

30498.562618570573
Mean Square Error for testing set 930162321.7988704
-0.7922957491887423


In [32]:
polynomial = PolynomialFeatures(degree=2)

XPolynomial = polynomial.fit_transform(XTrain)

Model = linear_model.ElasticNet()
Model.fit(XPolynomial, YTrain)

TrainPrediction = Model.predict(XPolynomial)

TesTPrediction = Model.predict(polynomial.fit_transform(XTest))

print(np.sqrt(mean_squared_error(YTest, TesTPrediction)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, TesTPrediction))
print(r2_score(YTest, TesTPrediction))

6550.56078868544
Mean Square Error for testing set 42909846.646263205
0.9173185862939436


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 4.267e+12, tolerance: 9.882e+09
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive


In [34]:
polynomial = PolynomialFeatures(degree=2)

XPolynomial = polynomial.fit_transform(XTrain)

Model = linear_model.Ridge()
Model.fit(XPolynomial, YTrain)

TrainPrediction = Model.predict(XPolynomial)

TesTPrediction = Model.predict(polynomial.fit_transform(XTest))

print(np.sqrt(mean_squared_error(YTest, TesTPrediction)))
print('Mean Square Error for testing set', metrics.mean_squared_error(YTest, TesTPrediction))
print(r2_score(YTest, TesTPrediction))

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_ridge.py:157: LinAlgWarning: Ill-conditioned matrix (rcond=1.31382e-20): result may not be accurate.
  return linalg.solve(A, Xy, sym_pos=True, overwrite_a=True).T


5566.156632389981
Mean Square Error for testing set 30982099.656298973
0.9403017256089914


In [33]:
df.head()

,ch_code,num_code,stop,type,price,month,day,dayofyear,dep_hour,dep_minute,hours_taken,minutes_taken,arr_hour,arr_minute,source,destination
0,7,812,1,0,56588,5,3,123,9,45,10.0,10.0,19,55,0,3
1,7,975,1,0,59649,3,18,77,5,45,6.0,30.0,12,15,2,3
2,3,7537,1,1,3916,9,3,246,14,30,8.0,10.0,22,40,5,3
3,3,287,1,1,6031,3,15,74,10,40,9.0,40.0,20,20,5,2
4,7,826,1,1,7081,3,22,81,12,30,7.0,25.0,19,55,1,4
